# Experiment Results

This page presents the results of a overall runtime of one week for each grammar.

## Details

- generation of inputs with [tribble](https://github.com/havrikov/tribble/)
- inputs generated from two grammars, one based on the current [URL standard](https://url.spec.whatwg.org/), the other one based on the RFC documents [RFC 3986](https://tools.ietf.org/html/rfc3986#appendix-A) and [RFC 6874](https://tools.ietf.org/html/rfc6874)
- experiment executed for both grammars with identical docker images
- for each run: 10 inputs selected at random from the generated inputs and added to the tests to execute
- run x contains all inputs used in runs 0..x-1 and 10 newly selected inputs
- execution of tests for 11 URL parsers:
    

*Language* | *URL Parser* | *Coverage Tool* 
--- | --- | ---
Firefox | nsURLParsers.cpp | grcov/genhtml
Chromium | url_parse.cc | coverage.py script
C | uriparser | LCOV 
C++ | POCO |LCOV 
Go | Package net/url | Package testing with coverage flags 
Java | java.net URL Class | JCOV 
JavaScript <br> | urijs <br> jsdom/whatwg-url| nyc/istanbul <br>
PHP | League URI | PHPUnit  
Python 3| urllib.parse | Coverage.py  
Ruby | Module URI |SimpleCov 

In [ ]:
import sys
!{sys.executable} -m pip install pandas markdown matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

ls_df=pd.read_csv("./livingstandard/coverages.csv", index_col='Run')

rfc_df=pd.read_csv("./rfc/coverages.csv", index_col='Run')
pd.set_option('display.float_format', lambda x: '%.3f' % x)

parsers=ls_df.columns.values
colors_all = ['orange', 'darkblue', 'red', 'yellow', 'lightblue', 'purple', 'brown', 'grey', 'black', 'pink','green']
colors_comp = ['darkgreen', 'blue']
ms=22


## Living Standard Results

### Tabular View

In [ ]:
ls_df.head(10)

In [ ]:
ls_df.tail(10)

### Overview Plot

In [ ]:

fig, axs=plt.subplots(figsize=(40,20))
plt.rcParams['font.size']='40'
plot=ls_df.plot(title="Living Standard Coverages Overview\n",ylim=(0,100),xlim=(-5,500), ax=axs,  color=colors_all, style=".")
plot.legend(bbox_to_anchor=(1,-0.2),markerscale=7.)
plot.grid(True)



axs.set_ylabel("Coverages", fontsize=40)
axs.set_xlabel("Run", fontsize=40)



### Detail Plot of the first 100 Runs

In [ ]:
fig, axs=plt.subplots(figsize=(40,20))
plt.rcParams['font.size']='40'
plot=ls_df.plot(title="Living Standard Coverages 100 Runs\n",ylim=(0,100),xlim=(-5,100), ax=axs, style=".", ms=ms, color=colors_all)
plot.legend(bbox_to_anchor=(1,-0.2),markerscale=2.)
plot.grid(True)



axs.set_ylabel("Coverages", fontsize=40)
axs.set_xlabel("Run", fontsize=40)



### Statistics

In [ ]:
ls_df.describe()

## RFC Results

### Tabular View

In [ ]:
rfc_df.head(10)

In [ ]:
rfc_df.tail(10)

### Overview Plot

In [ ]:
fig, axs=plt.subplots(figsize=(40,20))
plt.rcParams['font.size']='40'
plot=rfc_df.plot(title="RFC Coverages Overview\n",ylim=(0,100),xlim=(-5,500), ax=axs, style=".", color=colors_all)
plot.legend(bbox_to_anchor=(1,-0.2),markerscale=7.)
plot.grid(True)



axs.set_ylabel("Coverages", fontsize=40)
axs.set_xlabel("Run", fontsize=40)

### Detail Plot of the first 100 Runs

In [ ]:
fig, axs=plt.subplots(figsize=(40,20))
plt.rcParams['font.size']='40'
plot=rfc_df.plot(title="RFC Coverages 100 Runs\n",ylim=(0,100),xlim=(-5,100), ax=axs, style=".", ms=ms, color=colors_all)
plot.legend(bbox_to_anchor=(1,-0.2),markerscale=2.)
plot.grid(True)



axs.set_ylabel("Coverages", fontsize=40)
axs.set_xlabel("Run", fontsize=40)



### Statistics

In [ ]:
rfc_df.describe()

## Direct Comparison for each used Parser


In [ ]:
from IPython.display import display, Markdown

ffexcov=74.4
ffwptcov=84.4
chrexcov=83.65
chrwptcov=64.29


def compareGrammarResults(parser, df1, newcolumn1, df2, newcolumn2):
    display(Markdown("### "+parser.capitalize()))
    fig, axs=plt.subplots(figsize=(40,20))
    plt.rcParams['font.size']='40'

    new_df=pd.DataFrame()
    new_df[newcolumn1]=df1[parser]
    new_df[newcolumn2]=df2[parser]

    plot=new_df.plot(title=parser.capitalize()+" Coverages\n",ylim=(0,100),xlim=(-5,500), ax=axs, style=".", color=colors_comp, ms=ms)
    
    plot.grid(True)
    

    axs.set_ylabel("Coverages", fontsize=40)
    axs.set_xlabel("Run", fontsize=40)
    otherTests=""
    
    if parser=='firefox':
        plot.axhline(y=ffexcov, color='black', linestyle='-', lw=4, label='Existing Test Files')
        otherTests+="Existing Test Files Coverage: "+str(ffexcov)+"%"
        plot.axhline(y=ffwptcov, color='y', linestyle='-', lw=4, label='WPT tests')
        otherTests+="\nWeb platform tests Coverage: "+str(ffwptcov)+"%"
    if parser=='chromium':
        plot.axhline(y=chrexcov, color='black', linestyle='-', lw=4, label='Existing Test Files')
        otherTests+="Existing Test Files Coverage: "+str(chrexcov)+"%"
        plot.axhline(y=chrwptcov, color='y', linestyle='-', lw=4, label='WPT tests')
        otherTests+="\nWeb platform tests Coverage: "+str(chrwptcov)+"%"
    plot.legend(loc='best',markerscale=2.)
    plt.show()
    display(new_df.describe())

    print(newcolumn1+" Max Coverage: "+str(new_df[newcolumn1].max())+"%")
    print(newcolumn1+" Max Coverage reached in: run "+str(new_df[newcolumn1].idxmax()))
    print(newcolumn2+" Max Coverage: "+str(new_df[newcolumn2].max())+"%")
    print(newcolumn2+" Coverage reached in: run "+str(new_df[newcolumn2].idxmax()))
    print(otherTests)
    return new_df


In [ ]:
dfs={}

for parser in parsers:
    dfs[parser]=compareGrammarResults(parser, ls_df, "Living Standard", rfc_df, "RFC")


